##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 推荐电影：TFX 中的推荐模型

注：我们建议在 Colab 笔记本中运行本教程，无需进行设置！只需点击“在 Google Colab 中运行”。

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://tensorflow.google.cn/tfx/tutorials/tfx/recommenders"> <img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 TensorFlow.org 上查看</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/tfx/tutorials/tfx/recommenders.ipynb"> <img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 中运行</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/tfx/tutorials/tfx/recommenders.ipynb"> <img width="32px" src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 GitHub 上查看源代码</a></td>
<td><a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/tfx/tutorials/tfx/recommenders.ipynb"> <img width="32px" src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a></td>
</table></div>

## 移植到 TFX 的 TFRS 教程

下面是基本 TensorFlow Recommenders (TFRS) 教程到 TFX 的移植版，旨在演示如何在 TFX 流水线中使用 TFRS。它是[基本教程](https://tensorflow.google.cn/recommenders/examples/basic_retrieval)的镜像版。

对于上下文，现实世界的推荐系统通常由两个阶段组成：

1. 检索阶段负责从所有可能的候选项中选择数百个候选初始集。此模型的主要目标是有效剔除用户不感兴趣的所有候选项。由于检索模型可能要处理数百万个候选项，在计算上必须高效。
2. 排名阶段获取检索模型的输出并对其进行微调，以选择最佳的少量推荐。它的任务是将用户可能感兴趣的条目集缩小到可能的候选项名单。

在本教程中，我们将重点关注第一阶段“检索”。检索模型通常由两个子模型组成：

1. 使用查询特征计算查询表示（通常是固定维度嵌入向量）的查询模型。
2. 使用候选特征计算候选项表示（大小相等的向量）的候选项模型

然后，将这两个模型的输出相乘以给出查询-候选项亲和度分数，分数越高表示候选项与查询之间的匹配度越高。

在本教程中，我们将使用 Movielens 数据集构建和训练这样的一个双塔模型。

我们要完成以下步骤：

1. 注入并检查 MovieLens 数据集。
2. 实现检索模型。
3. 训练和导出模型。
4. 进行预测

## 数据集

Movielens 数据集是来自明尼苏达大学 [GroupLens](https://grouplens.org/datasets/movielens/) 研究小组的经典数据集。它包含一组用户对电影的评分，是推荐系统研究的主力。

可以通过两种方式处理数据：

1. 它可以被解释为表达用户观看（和评分）的电影，以及他们没有观看的电影。这是一种隐式反馈形式，用户的观看行为会告诉我们他们喜欢看哪些内容以及不喜欢看哪些内容。
2. 此外，它也可以被视为表达了用户对他们看过的电影的喜爱程度。这是一种显式反馈形式：假设用户观看了一部电影，我们可以通过查看他们给出的评分来大致判断他们的喜欢程度。

在本教程中，我们将重点关注检索系统：一个从目录中预测用户可能观看的一组电影的模型。通常，隐式数据在这里更有用，因此我们将把 Movielens 视为一个隐式系统。这意味着用户观看的每一部电影都是一个正样本，而他们没有看过的每一部电影都是一个隐式负样本。

## 导入

我们首先进行导入。

In [ ]:
!pip install -Uq tfx
!pip install -Uq tensorflow-recommenders
!pip install -Uq tensorflow-datasets

### 卸载 Shapely

TODO(b/263441833) 这是避免 ImportError 的临时解决方案。最终，应该通过支持最新版本的 Bigquery 来处理，而不是卸载其他额外的依赖项。

In [ ]:
!pip uninstall shapely -y

### 是否已重新启动运行时？

如果您使用的是 Google Colab，首次运行上面的代码单元时，必须重新启动运行时 (Runtime &gt; Restart runtime ...)。这样做的原因是 Colab 加载软件包的方式。

In [ ]:
import os
import absl
import json
import pprint
import tempfile

from typing import Any, Dict, List, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
import apache_beam as beam

from absl import logging

from tfx.components.example_gen.base_example_gen_executor import BaseExampleGenExecutor
from tfx.components.example_gen.component import FileBasedExampleGen
from tfx.components.example_gen import utils
from tfx.dsl.components.base import executor_spec

from tfx.types import artifact
from tfx.types import artifact_utils
from tfx.types import channel
from tfx.types import standard_artifacts
from tfx.types.standard_artifacts import Examples

from tfx.dsl.component.experimental.annotations import InputArtifact
from tfx.dsl.component.experimental.annotations import OutputArtifact
from tfx.dsl.component.experimental.annotations import Parameter
from tfx.dsl.component.experimental.decorators import component
from tfx.types.experimental.simple_artifacts import Dataset

from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

# Set up logging.
tf.get_logger().propagate = False
absl.logging.set_verbosity(absl.logging.INFO)
pp = pprint.PrettyPrinter()

print(f"TensorFlow version: {tf.__version__}")
print(f"TFX version: {tfx.__version__}")
print(f"TensorFlow Recommenders version: {tfrs.__version__}")

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

## 创建 TFDS ExampleGen

我们创建一个[自定义 ExampleGen 组件](https://tensorflow.google.cn/tfx/guide/examplegen#custom_examplegen)，用于加载 TensorFlow Datasets (TFDS) 数据集。它使用 FileBasedExampleGen 中的自定义执行器。

In [ ]:
@beam.ptransform_fn
@beam.typehints.with_input_types(beam.Pipeline)
@beam.typehints.with_output_types(tf.train.Example)
def _TFDatasetToExample(  # pylint: disable=invalid-name
    pipeline: beam.Pipeline,
    exec_properties: Dict[str, Any],
    split_pattern: str
    ) -> beam.pvalue.PCollection:
    """Read a TensorFlow Dataset and create tf.Examples"""
    custom_config = json.loads(exec_properties['custom_config'])
    dataset_name = custom_config['dataset']
    split_name = custom_config['split']

    builder = tfds.builder(dataset_name)
    builder.download_and_prepare()

    return (pipeline
            | 'MakeExamples' >> tfds.beam.ReadFromTFDS(builder, split=split_name)
            | 'AsNumpy' >> beam.Map(tfds.as_numpy)
            | 'ToDict' >> beam.Map(dict)
            | 'ToTFExample' >> beam.Map(utils.dict_to_example)
            )

class TFDSExecutor(BaseExampleGenExecutor):
  def GetInputSourceToExamplePTransform(self) -> beam.PTransform:
    """Returns PTransform for TF Dataset to TF examples."""
    return _TFDatasetToExample

## 初始化 TFX 流水线上下文

In [ ]:
context = InteractiveContext()

## 准备数据集

我们将在 `FileBasedExampleGen` 中使用自定义执行器从 TFDS 加载数据集。由于我们有两个数据集，将创建两个 `ExampleGen` 组件。

In [ ]:
# Ratings data.
ratings_example_gen = FileBasedExampleGen(
    input_base='dummy',
    custom_config={'dataset':'movielens/100k-ratings', 'split':'train'},
    custom_executor_spec=executor_spec.ExecutorClassSpec(TFDSExecutor))
context.run(ratings_example_gen, enable_cache=True)

In [ ]:
# Features of all the available movies.
movies_example_gen = FileBasedExampleGen(
    input_base='dummy',
    custom_config={'dataset':'movielens/100k-movies', 'split':'train'},
    custom_executor_spec=executor_spec.ExecutorClassSpec(TFDSExecutor))
context.run(movies_example_gen, enable_cache=True)

## 创建 `inspect_examples` 效用函数

我们创建一个方便的效用函数来检查 TF.Examples 的数据集。评分数据集返回一个包含电影 ID、用户 ID、指定评分、时间戳、电影信息和用户信息的字典：

In [ ]:
def inspect_examples(component,
                     channel_name='examples',
                     split_name='train',
                     num_examples=1):
  # Get the URI of the output artifact, which is a directory
  full_split_name = 'Split-{}'.format(split_name)
  print('channel_name: {}, split_name: {} (\"{}\"), num_examples: {}\n'.format(
      channel_name, split_name, full_split_name, num_examples))
  train_uri = os.path.join(
      component.outputs[channel_name].get()[0].uri, full_split_name)

  # Get the list of files in this directory (all compressed TFRecord files)
  tfrecord_filenames = [os.path.join(train_uri, name)
                        for name in os.listdir(train_uri)]

  # Create a `TFRecordDataset` to read these files
  dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

  # Iterate over the records and print them
  for tfrecord in dataset.take(num_examples):
    serialized_example = tfrecord.numpy()
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    pp.pprint(example)

inspect_examples(ratings_example_gen)

电影数据集包含电影 ID、电影标题和有关其所属类型的数据。请注意，类型是用整数标签编码的。

In [ ]:
inspect_examples(movies_example_gen)

## ExampleGen 进行拆分

当我们提取电影镜头数据集时，我们的 `ExampleGen` 组件将数据拆分为 `train` 和 `eval` 两部分。它们实际上被命名为 `Split-train` 和 `Split-eval`。默认情况下，66% 为训练，34% 为评估。

## 为电影和评分生成统计数据

对于 TFX 流水线，我们需要为数据集生成统计数据。我们通过使用 [StatisticsGen 组件](https://tensorflow.google.cn/tfx/guide/statsgen)来实现此目的。当我们为数据集生成模式时，下面的 [SchemaGen 组件](https://tensorflow.google.cn/tfx/guide/schemagen)将使用这些统计数据。无论如何，这是一种不错的做法，因为持续检查和分析数据至关重要。由于我们有两个数据集，将创建两个 StatisticsGen 组件。

In [ ]:
movies_stats_gen = tfx.components.StatisticsGen(
    examples=movies_example_gen.outputs['examples'])
context.run(movies_stats_gen, enable_cache=True)

In [ ]:
context.show(movies_stats_gen.outputs['statistics'])

In [ ]:
ratings_stats_gen = tfx.components.StatisticsGen(
    examples=ratings_example_gen.outputs['examples'])
context.run(ratings_stats_gen, enable_cache=True)

In [ ]:
context.show(ratings_stats_gen.outputs['statistics'])

## 为电影和评分创建模式

对于 TFX 流水线，我们需要从数据集中生成数据模式。我们通过使用 [SchemaGen 组件](https://tensorflow.google.cn/tfx/guide/schemagen)来实现此目的。下面的 [Transform 组件](https://tensorflow.google.cn/tfx/guide/transform)将使用此数据模式以一种高度可扩展到大型数据集的方式执行特征工程，并避免训练/提供偏差。由于我们有两个数据集，将创建两个 SchemaGen 组件。

In [ ]:
movies_schema_gen = tfx.components.SchemaGen(
    statistics=movies_stats_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(movies_schema_gen, enable_cache=True)

In [ ]:
context.show(movies_schema_gen.outputs['schema'])

In [ ]:
ratings_schema_gen = tfx.components.SchemaGen(
    statistics=ratings_stats_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(ratings_schema_gen, enable_cache=True)

In [ ]:
context.show(ratings_schema_gen.outputs['schema'])

## 使用 Transform 执行特征工程

对于 TFX 流水线的结构化和可重复设计，我们需要一种可扩展的特征工程方法。这样我们便能处理通常是许多推荐系统一部分的大型数据集，并且还避免了训练/应用偏差。我们将使用 [Transform 组件](https://tensorflow.google.cn/tfx/guide/transform)来实现此目的。

Transform 组件使用模块文件为我们想要执行的特征工程提供用户代码，因此我们的第一步是创建该模块文件。由于我们有两个数据集，将创建其中两个模块文件和两个 Transform 组件。

我们的推荐系统需要的内容之一是 `user_id` 和 `movie_title` 字段的词汇表。在 [basic_retrieval 教程](https://tensorflow.google.cn/recommenders/examples/basic_retrieval)中，它们是使用内嵌 Numpy 创建的，但在这里我们将使用 Transform。

注：下面的 `%%writefile {_movies_transform_module_file}` 单元魔法会创建该单元的内容并将其写入运行此笔记本的笔记本服务器（例如，Colab VM）上的文件中。在笔记本之外执行此操作时，只需创建一个 Python 文件。

In [ ]:
_movies_transform_module_file = 'movies_transform_module.py'

In [ ]:
%%writefile {_movies_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

def preprocessing_fn(inputs):
  # We only want the movie title
  return {'movie_title':inputs['movie_title']}

In [ ]:
movies_transform = tfx.components.Transform(
    examples=movies_example_gen.outputs['examples'],
    schema=movies_schema_gen.outputs['schema'],
    module_file=os.path.abspath(_movies_transform_module_file))
context.run(movies_transform, enable_cache=True)

In [ ]:
context.show(movies_transform.outputs['post_transform_schema'])

In [ ]:
inspect_examples(movies_transform, channel_name='transformed_examples')

In [ ]:
_ratings_transform_module_file = 'ratings_transform_module.py'

In [ ]:
%%writefile {_ratings_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft
import pdb

NUM_OOV_BUCKETS = 1

def preprocessing_fn(inputs):
  # We only want the user ID and the movie title, but we also need vocabularies
  # for both of them.  The vocabularies aren't features, they're only used by
  # the lookup.
  outputs = {}
  outputs['user_id'] = tft.sparse_tensor_to_dense_with_shape(inputs['user_id'], [None, 1], '-1')
  outputs['movie_title'] = tft.sparse_tensor_to_dense_with_shape(inputs['movie_title'], [None, 1], '-1')

  tft.compute_and_apply_vocabulary(
      inputs['user_id'],
      num_oov_buckets=NUM_OOV_BUCKETS,
      vocab_filename='user_id_vocab')

  tft.compute_and_apply_vocabulary(
      inputs['movie_title'],
      num_oov_buckets=NUM_OOV_BUCKETS,
      vocab_filename='movie_title_vocab')

  return outputs

In [ ]:
ratings_transform = tfx.components.Transform(
    examples=ratings_example_gen.outputs['examples'],
    schema=ratings_schema_gen.outputs['schema'],
    module_file=os.path.abspath(_ratings_transform_module_file))
context.run(ratings_transform, enable_cache=True)

In [ ]:
context.show(ratings_transform.outputs['post_transform_schema'])

In [ ]:
inspect_examples(ratings_transform, channel_name='transformed_examples')

## 在 TFX 中实现模型

在 [basic_retrieval](https://tensorflow.google.cn/recommenders/examples/basic_retrieval) 教程中，模型是在 Python 运行时中以内嵌方式创建的。在 TFX 流水线中，模型、指标和损失在[名为 Trainer 的流水线组件](https://tensorflow.google.cn/tfx/guide/trainer)的模块文件中定义和训练。这可让模型、指标和损失成为可自动化和受监视的可重复过程的一部分。

### TensorFlow Recommenders 模型架构

我们将构建一个双塔检索模型。双塔的概念意味着我们将拥有一个使用用户特征计算用户表示的查询塔，以及另一个使用电影特征计算电影表示的条目塔。我们可以单独构建每个塔（在下面的 `_build_user_model()` 和 `_build_movie_model()` 方法中），然后将它们组合到最终模型中（如在 `MobieLensModel` 类中）。`MovieLensModel` 是 `tfrs.Model` 基类（可简化模型构建）的子类：我们需要做的就是在 `__init__` 方法中设置组件，并实现 `compute_loss` 方法，获取原始特征并返回损失值。

In [ ]:
# We're now going to create the module file for Trainer, which will include the
# code above with some modifications for TFX.

_trainer_module_file = 'trainer_module.py'

In [ ]:
%%writefile {_trainer_module_file}

from typing import Dict, List, Text

import pdb

import os
import absl
import datetime
import glob
import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_recommenders as tfrs

from absl import logging
from tfx.types import artifact_utils

from tfx import v1 as tfx
from tfx_bsl.coders import example_coder
from tfx_bsl.public import tfxio

absl.logging.set_verbosity(absl.logging.INFO)

EMBEDDING_DIMENSION = 32
INPUT_FN_BATCH_SIZE = 1


def extract_str_feature(dataset, feature_name):
  np_dataset = []
  for example in dataset:
    np_example = example_coder.ExampleToNumpyDict(example.numpy())
    np_dataset.append(np_example[feature_name][0].decode())
  return tf.data.Dataset.from_tensor_slices(np_dataset)


class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model, tf_transform_output, movies_uri):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model

    movies_artifact = movies_uri.get()[0]
    input_dir = artifact_utils.get_split_uri([movies_artifact], 'train')
    movie_files = glob.glob(os.path.join(input_dir, '*'))
    movies = tf.data.TFRecordDataset(movie_files, compression_type="GZIP")
    movies_dataset = extract_str_feature(movies, 'movie_title')

    loss_metrics = tfrs.metrics.FactorizedTopK(
        candidates=movies_dataset.batch(128).map(movie_model)
        )

    self.task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=loss_metrics
        )


  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    try:
      user_embeddings = tf.squeeze(self.user_model(features['user_id']), axis=1)
      # And pick out the movie features and pass them into the movie model,
      # getting embeddings back.
      positive_movie_embeddings = self.movie_model(features['movie_title'])

      # The task computes the loss and the metrics.
      _task = self.task(user_embeddings, positive_movie_embeddings)
    except BaseException as err:
      logging.error('######## ERROR IN compute_loss:\n{}\n###############'.format(err))

    return _task


# This function will apply the same transform operation to training data
# and serving requests.
def _apply_preprocessing(raw_features, tft_layer):
  try:
    transformed_features = tft_layer(raw_features)
  except BaseException as err:
    logging.error('######## ERROR IN _apply_preprocessing:\n{}\n###############'.format(err))

  return transformed_features


def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  try:
    return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size),
      tf_transform_output.transformed_metadata.schema)
  except BaseException as err:
    logging.error('######## ERROR IN _input_fn:\n{}\n###############'.format(err))

  return None


def _get_serve_tf_examples_fn(model, tf_transform_output):
  """Returns a function that parses a serialized tf.Example and applies TFT."""
  try:
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
      """Returns the output to be used in the serving signature."""
      try:
        feature_spec = tf_transform_output.raw_feature_spec()
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        transformed_features = model.tft_layer(parsed_features)
        result = model(transformed_features)
      except BaseException as err:
        logging.error('######## ERROR IN serve_tf_examples_fn:\n{}\n###############'.format(err))
      return result
  except BaseException as err:
      logging.error('######## ERROR IN _get_serve_tf_examples_fn:\n{}\n###############'.format(err))

  return serve_tf_examples_fn


def _build_user_model(
    tf_transform_output: tft.TFTransformOutput, # Specific to ratings
    embedding_dimension: int = 32) -> tf.keras.Model:
  """Creates a Keras model for the query tower.

  Args:
    tf_transform_output: [tft.TFTransformOutput], the results of Transform
    embedding_dimension: [int], the dimensionality of the embedding space

  Returns:
    A keras Model.
  """
  try:
    unique_user_ids = tf_transform_output.vocabulary_by_name('user_id_vocab')
    users_vocab_str = [b.decode() for b in unique_user_ids]

    model = tf.keras.Sequential(
        [
         tf.keras.layers.StringLookup(
             vocabulary=users_vocab_str, mask_token=None),
         # We add an additional embedding to account for unknown tokens.
         tf.keras.layers.Embedding(len(users_vocab_str) + 1, embedding_dimension)
         ])
  except BaseException as err:
    logging.error('######## ERROR IN _build_user_model:\n{}\n###############'.format(err))

  return model


def _build_movie_model(
    tf_transform_output: tft.TFTransformOutput, # Specific to movies
    embedding_dimension: int = 32) -> tf.keras.Model:
  """Creates a Keras model for the candidate tower.

  Args:
    tf_transform_output: [tft.TFTransformOutput], the results of Transform
    embedding_dimension: [int], the dimensionality of the embedding space

  Returns:
    A keras Model.
  """
  try:
    unique_movie_titles = tf_transform_output.vocabulary_by_name('movie_title_vocab')
    titles_vocab_str = [b.decode() for b in unique_movie_titles]

    model = tf.keras.Sequential(
        [
         tf.keras.layers.StringLookup(
             vocabulary=titles_vocab_str, mask_token=None),
         # We add an additional embedding to account for unknown tokens.
         tf.keras.layers.Embedding(len(titles_vocab_str) + 1, embedding_dimension)
        ])
  except BaseException as err:
      logging.error('######## ERROR IN _build_movie_model:\n{}\n###############'.format(err))
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  try:
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

    train_dataset = _input_fn(fn_args.train_files, fn_args.data_accessor,
                              tf_transform_output, INPUT_FN_BATCH_SIZE)
    eval_dataset = _input_fn(fn_args.eval_files, fn_args.data_accessor,
                            tf_transform_output, INPUT_FN_BATCH_SIZE)

    model = MovielensModel(
        _build_user_model(tf_transform_output, EMBEDDING_DIMENSION),
        _build_movie_model(tf_transform_output, EMBEDDING_DIMENSION),
        tf_transform_output,
        fn_args.custom_config['movies']
        )

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=fn_args.model_run_dir, update_freq='batch')

    model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))
  except BaseException as err:
    logging.error('######## ERROR IN run_fn before fit:\n{}\n###############'.format(err))

  try:
    model.fit(
        train_dataset,
        epochs=fn_args.custom_config['epochs'],
        steps_per_epoch=fn_args.train_steps,
        validation_data=eval_dataset,
        validation_steps=fn_args.eval_steps,
        callbacks=[tensorboard_callback])
  except BaseException as err:
      logging.error('######## ERROR IN run_fn during fit:\n{}\n###############'.format(err))

  try:
    index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)

    movies_artifact = fn_args.custom_config['movies'].get()[0]
    input_dir = artifact_utils.get_split_uri([movies_artifact], 'eval')
    movie_files = glob.glob(os.path.join(input_dir, '*'))
    movies = tf.data.TFRecordDataset(movie_files, compression_type="GZIP")

    movies_dataset = extract_str_feature(movies, 'movie_title')

    index.index_from_dataset(
      tf.data.Dataset.zip((
          movies_dataset.batch(100),
          movies_dataset.batch(100).map(model.movie_model))
      )
    )

    # Run once so that we can get the right signatures into SavedModel
    _, titles = index(tf.constant(["42"]))
    print(f"Recommendations for user 42: {titles[0, :3]}")

    signatures = {
        'serving_default':
            _get_serve_tf_examples_fn(index,
                                      tf_transform_output).get_concrete_function(
                                          tf.TensorSpec(
                                              shape=[None],
                                              dtype=tf.string,
                                              name='examples')),
    }
    index.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

  except BaseException as err:
      logging.error('######## ERROR IN run_fn during export:\n{}\n###############'.format(err))

## 训练模型

定义模型后，我们可以运行 [Trainer 组件](https://tensorflow.google.cn/tfx/guide/trainer)来进行模型训练。

In [ ]:
trainer = tfx.components.Trainer(
    module_file=os.path.abspath(_trainer_module_file),
    examples=ratings_transform.outputs['transformed_examples'],
    transform_graph=ratings_transform.outputs['transform_graph'],
    schema=ratings_transform.outputs['post_transform_schema'],
    train_args=tfx.proto.TrainArgs(num_steps=500),
    eval_args=tfx.proto.EvalArgs(num_steps=10),
    custom_config={
        'epochs':5,
        'movies':movies_transform.outputs['transformed_examples'],
        'movie_schema':movies_transform.outputs['post_transform_schema'],
        'ratings':ratings_transform.outputs['transformed_examples'],
        'ratings_schema':ratings_transform.outputs['post_transform_schema']
        })

context.run(trainer, enable_cache=False)

## 导出模型

训练模型后，我们可以使用 [Pusher 组件](https://tensorflow.google.cn/tfx/guide/pusher)导出模型。

In [ ]:
_serving_model_dir = os.path.join(tempfile.mkdtemp(), 'serving_model/tfrs_retrieval')

pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
    push_destination=tfx.proto.PushDestination(
        filesystem=tfx.proto.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher, enable_cache=True)

## 进行预测

现在我们获得了一个模型，我们将其加载回来并进行预测。

In [ ]:
loaded = tf.saved_model.load(pusher.outputs['pushed_model'].get()[0].uri)
scores, titles = loaded(["42"])

print(f"Recommendations: {titles[0][:3]}")

## 后续步骤

在本教程中，您学习了如何使用 TensorFlow Recommenders 和 TFX 实现检索模型。要扩展此处介绍的内容，请查阅[使用 TFX 进行 TFRS 排名](https://tensorflow.google.cn/recommenders/examples/ranking_tfx)教程。